In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List
import numpy as np
from pathlib import Path
from matplotlib import ticker



In [ ]:
class BlockRelocationAnalyzer:
    def __init__(self, results_path: str):
        with open(results_path, 'r') as f:
            self.raw_data = json.load(f)
            
            self.warehouse_info = self.raw_data['warehouse']
            
        self.warehouse_str = f"Warehouse {self.warehouse_info['dimensions']}"
        self.snapshots_df = self._process_snapshots()
        self.runs_df = self._process_runs()
            
        # Set better plotting style
        plt.style.use('seaborn-v0_8-whitegrid')
        plt.rcParams.update({
            'font.family': 'serif',
            'font.size': 10,
            'figure.figsize': [20, 20],
            'axes.titlesize': 11,
            'axes.labelsize': 10,
            'legend.fontsize': 9,
            'legend.framealpha': 1.0,
            'legend.facecolor': 'white',
            'legend.edgecolor': 'gray'
        })

    def _process_snapshots(self) -> pd.DataFrame:
        records = []
        # Create event type mapping
        event_type_map = {
            0: 'ExpectedExecutionEvent',
            1: 'NewBlockEvent',
            2: 'MissmoveEvent',
            3: 'BlockTargetUpdateEvent'
        }
        
        for run in self.raw_data['runs']:
            seed = run['seed']
            for config_id, result in run['results'].items():
                variant = next(c['variant'] for c in self.raw_data['configs'] 
                            if c['id'] == config_id)
                
                for snapshot in result['snapshots']:
                    record = {
                        'seed': seed,
                        'variant': variant,
                        'event_number': snapshot['eventNumber'],
                        'total_cost_so_far': snapshot.get('totalCostSoFar', 0),
                        'recalculation_time': snapshot.get('recalculationTimeMs'),
                        'event_type': event_type_map[snapshot['eventType']],  # Map the numeric event type to string
                        'warehouse_utilization': snapshot['warehouseUtilization'],
                        'arrival_queue_size': snapshot['arrivalStackHeight'],
                        'blocked_blocks': snapshot['blockedBlocks'],
                        'empty_stacks': snapshot['emptyStacks'],
                        'planned_moves': snapshot['plannedMovesRemaining'],
                        'bound_change': snapshot.get('boundChange', 0)  # Get bound change directly from snapshot
                    }
                    records.append(record)
        return pd.DataFrame(records)

    def _process_runs(self) -> pd.DataFrame:
        records = []
        for run in self.raw_data['runs']:
            seed = run['seed']
            for config_id, result in run['results'].items():
                variant = next(c['variant'] for c in self.raw_data['configs'] 
                            if c['id'] == config_id)
                # Get total events configured from configs
                total_events = next(c['totalEvents'] for c in self.raw_data['configs'] 
                                if c['id'] == config_id)
                
                # Calculate events reached percentage
                events_reached = sum(result['eventCounts'].values())
                events_reached_percentage = (events_reached / total_events) * 100
                
                record = {
                    'seed': seed,
                    'variant': variant,
                    'runtime': result['runtime'],
                    'total_cost': result['totalMoveCost'],
                    'total_moves': result['totalMovesExecuted'],
                    'averageRecalculationTime': result['averageRecalculationTime'],
                    'totalRecalculationTime': result['totalRecalculationTime'],
                    'totalMovesExecuted': result['totalMovesExecuted'],
                    'totalMoveCost': result['totalMoveCost'],
                    'recalculationCount': result['recalculationCount'],
                    'status': result['status'],
                    'events_reached': events_reached,
                    'events_reached_percentage': events_reached_percentage,
                    'total_events_configured': total_events
                }
                records.append(record)
        return pd.DataFrame(records)
    
    def generate_summary_stats(self, output_path: Path):
        """Generate and save summary statistics as CSV"""
        summary = self.runs_df.groupby('variant').agg({
            'runtime': ['mean', 'std', 'min', 'max'],
            'total_cost': ['mean', 'std', 'min', 'max'],
            'events_reached_percentage': ['mean', 'std', 'min', 'max'],
            'total_moves': ['mean', 'std', 'min', 'max']
        }).round(2)
        
        # Add status distribution
        status_dist = pd.crosstab(
            self.runs_df['variant'], 
            self.runs_df['status'], 
            normalize='index'
        ).round(3) * 100
        
        # Combine statistics
        for status in status_dist.columns:
            summary[('status_distribution', status)] = status_dist[status]
        
        # Save to CSV
        summary.to_csv(output_path / 'summary_statistics.csv')
        return summary

    def _format_variant_name(self, name: str) -> str:
        """Format variant names for better readability"""
        words = []
        current = ""
        for char in name:
            if char.isupper() and current:
                words.append(current)
                current = char
            else:
                current += char
        words.append(current)
        return ' '.join(words)

    def plot_analysis(self, save_path = None):
        """Generate comprehensive analysis plots"""
        fig = plt.figure(figsize=(30, 30))
        # Increase spacing between plots
        gs = plt.GridSpec(5, 2, figure=fig, hspace=0.5, wspace=0.4)
        
        # Get variants in original order from the config
        variants = [config['variant'] for config in self.raw_data['configs']]
        
        # Set up consistent colors and markers for variants
        colors = plt.cm.Set2(np.linspace(0, 1, len(variants)))
        markers = ['o', 's', '^', 'D', 'v', 'p', '*']
        variant_styles = {
            variant: {
                'color': color,
                'marker': marker
            }
            for variant, color, marker in zip(variants, colors, markers)
        }
        
        # 1. Total Cost vs Runtime
        ax1 = fig.add_subplot(gs[0, 0])
        for variant in variants:
            variant_data = self.runs_df[self.runs_df['variant'] == variant]
            style = variant_styles[variant]
            
            # Get successful runs only for plotting
            successful_data = variant_data[variant_data['status'] == 'Completed']
            if len(successful_data) > 0:
                ax1.scatter(
                    successful_data['runtime'],
                    successful_data['total_cost'],
                    color=style['color'],
                    marker=style['marker'],
                    s=100,
                    alpha=0.7
                )

            # Check for failed runs and add to legend if present
            failed_runs = variant_data[variant_data['status'] == 'Failed']
            if len(failed_runs) > 0:
                label = f"{self._format_variant_name(variant)} (Failed)"
            else:
                label = self._format_variant_name(variant)

            # terminated runs
            terminated_runs = variant_data[variant_data['status'] == 'Terminated']
            if len(terminated_runs) > 0:
                label = f"{self._format_variant_name(variant)} (Terminated)"
            else:
                label = self._format_variant_name(variant)
                
            # Add legend entry
            ax1.scatter([], [], # Empty scatter for legend
                label=label,
                color=style['color'],
                marker=style['marker'],
                s=100,
                alpha=0.7
            )

        ax1.set_title('Total Cost vs Runtime (by Status)')
        ax1.set_xlabel('Runtime (seconds)')
        ax1.set_ylabel('Total Cost')
        ax1.legend(bbox_to_anchor=(1.02, 1), fontsize=8, labelspacing=0.2)
        
        # 2. Cost Accumulation
        ax2 = fig.add_subplot(gs[0, 1])
        completed_snapshots = self.snapshots_df[
            self.snapshots_df['variant'].isin(
                self.runs_df[self.runs_df['status'] == 'Completed']['variant']
            )
        ]
        cost_by_event = completed_snapshots.groupby(['variant', 'event_number'])['total_cost_so_far'].mean().reset_index()
        
        for variant in variants:
            variant_data = cost_by_event[cost_by_event['variant'] == variant]
            if len(variant_data) > 0:  # Only plot if there's data
                style = variant_styles[variant]
                ax2.plot(
                    variant_data['event_number'],
                    variant_data['total_cost_so_far'],
                    label=f"{self._format_variant_name(variant)} (n={len(self.runs_df[(self.runs_df['variant'] == variant) & (self.runs_df['status'] == 'Completed')])})",
                    color=style['color'],
                    marker=style['marker'],
                    markevery=20,
                    markersize=6,
                    linewidth=2
                )
        ax2.set_title('Cost Accumulation Over Time (Completed Runs Only)')
        ax2.set_xlabel('Event Number')
        ax2.set_ylabel('Total Cost')
        ax2.legend(bbox_to_anchor=(1.02, 1), fontsize=8, labelspacing=0.2)
        
        # 3. Arrival Queue Size
        ax3 = fig.add_subplot(gs[1, 0])
        queue_by_event = self.snapshots_df.groupby(['variant', 'event_number'])['arrival_queue_size'].mean().reset_index()
        for variant in variants:
            variant_data = queue_by_event[queue_by_event['variant'] == variant]
            style = variant_styles[variant]
            ax3.plot(
                variant_data['event_number'],
                variant_data['arrival_queue_size'],
                label=self._format_variant_name(variant),
                color=style['color'],
                marker=style['marker'],
                markevery=20,
                markersize=6,
                linewidth=2
            )
        # Add horizontal line for allowed queue size
        allowed_queue_size = 3 
        ax3.axhline(y=allowed_queue_size, color='red', linestyle='--', linewidth=1.5, label='Allowed Size')
        
        ax3.set_title('Arrival Queue Size Evolution')
        ax3.set_xlabel('Event Number')
        ax3.set_ylabel('Queue Size')
        ax3.legend(bbox_to_anchor=(1.02, 1), fontsize=8, labelspacing=0.2)
        
        # 4. Empty Stacks
        ax4 = fig.add_subplot(gs[1, 1])
        empty_by_event = self.snapshots_df.groupby(['variant', 'event_number'])['empty_stacks'].mean().reset_index()
        for variant in variants:
            variant_data = empty_by_event[empty_by_event['variant'] == variant]
            style = variant_styles[variant]
            ax4.plot(
                variant_data['event_number'],
                variant_data['empty_stacks'],
                label=self._format_variant_name(variant),
                color=style['color'],
                marker=style['marker'],
                markevery=20,
                markersize=6,
                linewidth=2
            )
        ax4.set_title('Empty Stacks Evolution')
        ax4.set_xlabel('Event Number')
        ax4.set_ylabel('Number of Empty Stacks')
        ax4.legend(bbox_to_anchor=(1.02, 1), fontsize=8, labelspacing=0.2)
        
        # 5. Event Distribution
        ax5 = fig.add_subplot(gs[2, 0])

        # Define the desired column order
        desired_order = ['ExpectedExecutionEvent', 'NewBlockEvent', 'MissmoveEvent', 'BlockTargetUpdateEvent']

        event_counts = pd.crosstab(
            pd.Categorical(self.snapshots_df['variant'], categories=variants),
            self.snapshots_df['event_type'],
            normalize='index'
        ) * 100

        # Reorder the columns
        event_counts = event_counts[desired_order]

        event_colors = plt.cm.Set3(np.linspace(0, 1, 4))
        event_counts.plot(
            kind='bar',
            stacked=True,
            ax=ax5,
            width=0.8,
            color=event_colors
        )

        ax5.set_title('Event Type Distribution')
        ax5.set_xlabel('Variant')
        ax5.set_ylabel('Proportion (%)')
        ax5.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax5.get_xticklabels()],
            rotation=30,
            ha='right'
        )

        # Legend order will now match the stacking order
        ax5.legend(
            ['Expected Execution', 'New Block', 'Missmove', 'Block Target Update'],
            title='Event Type',
            bbox_to_anchor=(1.02, 1),
            fontsize=8,
            labelspacing=0.2
        )
        
        # 6. Recalculation Times
        ax6 = fig.add_subplot(gs[2, 1])
        recalc_data = self.snapshots_df[self.snapshots_df['recalculation_time'].notna()].copy()
        recalc_data['variant'] = pd.Categorical(recalc_data['variant'], categories=variants)
        palette = {variant: style['color'] for variant, style in variant_styles.items()}
        sns.boxplot(
            data=recalc_data,
            x='variant',
            y='recalculation_time',
            ax=ax6,
            palette=palette,
            showfliers=False,
            order=variants
        )
        ax6.set_title('Recalculation Time Distribution')
        ax6.set_xlabel('Variant')
        ax6.set_ylabel('Recalculation Time (ms)')
        ax6.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax6.get_xticklabels()],
            rotation=30,
            ha='right'
        )
        
        # 7. Planned Moves Evolution
        ax7 = fig.add_subplot(gs[3, 0])
        moves_by_event = self.snapshots_df.groupby(['variant', 'event_number'])['planned_moves'].mean().reset_index()
        for variant in variants:
            variant_data = moves_by_event[moves_by_event['variant'] == variant]
            style = variant_styles[variant]
            ax7.plot(
                variant_data['event_number'],
                variant_data['planned_moves'],
                label=self._format_variant_name(variant),
                color=style['color'],
                marker=style['marker'],
                markevery=20,
                markersize=6,
                linewidth=2
            )
        ax7.set_title('Planned Moves Evolution')
        ax7.set_xlabel('Event Number')
        ax7.set_ylabel('Number of Planned Moves')
        ax7.legend(bbox_to_anchor=(1.02, 1), fontsize=8, labelspacing=0.2)
        
        # 8. Warehouse Utilization
        ax8 = fig.add_subplot(gs[3, 1])
        util_by_event = self.snapshots_df.groupby(['variant', 'event_number'])['warehouse_utilization'].mean().reset_index()
        for variant in variants:
            variant_data = util_by_event[util_by_event['variant'] == variant]
            style = variant_styles[variant]
            ax8.plot(
                variant_data['event_number'],
                variant_data['warehouse_utilization'] * 100,
                label=self._format_variant_name(variant),
                color=style['color'],
                marker=style['marker'],
                markevery=20,
                markersize=6,
                linewidth=2
            )
        ax8.set_title('Warehouse Utilization Over Time')
        ax8.set_xlabel('Event Number')
        ax8.set_ylabel('Utilization (%)')
        ax8.legend(bbox_to_anchor=(1.02, 1), fontsize=8, labelspacing=0.2)

        ax9 = fig.add_subplot(gs[4, 1])  
        status_counts = pd.crosstab(
            pd.Categorical(self.runs_df['variant'], categories=variants),
            self.runs_df['status'],
            normalize='index'
        ) * 100
        status_colors = {'Completed': '#2ecc71', 'Failed': '#e74c3c', 'Terminated': '#f1c40f'}
        status_counts.plot(
            kind='bar',
            stacked=True,
            ax=ax9,
            color=[status_colors.get(x, '#95a5a6') for x in status_counts.columns],
            width=0.8
        )
        ax9.set_title('Simulation Status Distribution')
        ax9.set_xlabel('Variant')
        ax9.set_ylabel('Proportion (%)')
        ax9.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax9.get_xticklabels()],
            rotation=30,
            ha='right'
        )
        ax9.legend(title='Status', bbox_to_anchor=(1.02, 1), fontsize=8, labelspacing=0.2)


        # Get just the colors from variant_styles
        variant_colors = {variant: style['color'] for variant, style in variant_styles.items()}

        # Add new subplot for events reached
        ax_events = fig.add_subplot(gs[4, 0])  # Adjust grid as needed

        # Create boxplot for events reached percentage using just the colors
        sns.boxplot(
            data=self.runs_df,
            x='variant',
            y='events_reached_percentage',
            ax=ax_events,
            palette=variant_colors,  # Use the color dictionary instead
            showfliers=False
        )

        # Add individual points
        sns.stripplot(
            data=self.runs_df,
            x='variant',
            y='events_reached_percentage',
            ax=ax_events,
            color='black',
            alpha=0.4,
            size=4,
            jitter=0.2
        )

        ax_events.set_title('Percentage of Events Reached by Variant')
        ax_events.set_xlabel('Variant')
        ax_events.set_ylabel('Events Reached (%)')
        ax_events.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax_events.get_xticklabels()],
            rotation=30,
            ha='right'
        )

        
        # Adjust layout to prevent overlap
        plt.tight_layout(rect=[0, 0, 0.85, 1])  # Leave more space on the right for legends
        
        if save_path:
            plt.savefig(save_path / 'analysis.png', dpi=300, bbox_inches='tight', pad_inches=0.3)
            plt.close()
        else:
            plt.show()
        event_impact_fig = self.plot_event_impact()
        if save_path:
            event_impact_fig.savefig(save_path / 'event_impact.png', 
                                    dpi=300, bbox_inches='tight', pad_inches=0.3)
            plt.close(event_impact_fig)

    def plot_event_impact(self):
        """Create visualization showing bound impact per event type"""
        # Create figure with two subplots side by side
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
        
        variants = [config['variant'] for config in self.raw_data['configs']]
        event_types = ['NewBlockEvent', 'MissmoveEvent', 'BlockTargetUpdateEvent']
        
        # Calculate average bound change per event type for each variant
        impact_data = []
        for variant in variants:
            variant_snapshots = self.snapshots_df[self.snapshots_df['variant'] == variant]
            
            for event_type in event_types:
                avg_impact = variant_snapshots[
                    variant_snapshots['event_type'] == event_type
                ]['bound_change'].median()
                
                impact_data.append({
                    'variant': variant,
                    'event_type': event_type,
                    'avg_impact': avg_impact if not pd.isna(avg_impact) else 0
                })
        
        # Convert to DataFrame for easier plotting
        impact_df = pd.DataFrame(impact_data)
        
        # BAR PLOT
        # Set up bar positions
        x = np.arange(len(variants))
        width = 0.25  # Slightly wider bars since we have one less event type
        multiplier = 0
        
        # Plot bars for each event type
        for event_type in event_types:
            event_data = impact_df[impact_df['event_type'] == event_type]
            offset = width * multiplier
            rects = ax1.bar(x + offset, event_data['avg_impact'], width,
                        label=event_type.replace('Event', ''))
            multiplier += 1
        
        # Customize the bar plot
        ax1.set_ylabel('Median Bound Change')
        ax1.set_title('Median Bound Change by Event Type and Variant', pad=20, fontsize=12, fontweight='bold')
        ax1.set_xticks(x + width)
        ax1.set_xticklabels([self._format_variant_name(v) for v in variants],
                        rotation=45, ha='right')
        ax1.legend(title='Event Type', bbox_to_anchor=(1.02, 1))
        ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.3)
        
        # Add value labels on the bars
        for rect in ax1.patches:
            height = rect.get_height()
            ax1.annotate(f'{height:.1f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3 if height >= 0 else -3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom' if height >= 0 else 'top',
                    fontsize=8)

        # BOXPLOT
        # Prepare data for boxplot
        boxplot_data = []
        variant_labels = []
        event_labels = []
        
        for variant in variants:
            variant_snapshots = self.snapshots_df[self.snapshots_df['variant'] == variant]
            for event_type in event_types:
                bound_changes = variant_snapshots[
                    variant_snapshots['event_type'] == event_type
                ]['bound_change'].values
                boxplot_data.append(bound_changes)
                variant_labels.extend([self._format_variant_name(variant)] * len(bound_changes))
                event_labels.extend([event_type.replace('Event', '')] * len(bound_changes))
        
        # Create DataFrame for boxplot
        boxplot_df = pd.DataFrame({
            'Variant': variant_labels,
            'Event Type': event_labels,
            'Bound Change': np.concatenate(boxplot_data)
        })
        
        # Create boxplot
        sns.boxplot(data=boxplot_df, x='Variant', y='Bound Change', hue='Event Type', ax=ax2)
        ax2.set_title('Distribution of Bound Changes by Event Type', pad=20, fontsize=12, fontweight='bold')
        ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
        ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.3)
        
        # Adjust layout
        plt.suptitle('Analysis of Bound Changes by Event Type and Variant', 
                    fontsize=14, fontweight='bold', y=1.05)
        plt.tight_layout()
        return fig
            
    def analyze_correlations(self):
        """Calculate and print correlations between warehouse utilization, planned moves, and arrival queue size"""
        print("\nCorrelation Analysis:")
        print("-" * 50)
        
        # Get unique variants
        variants = self.snapshots_df['variant'].unique()
        
        for variant in variants:
            variant_data = self.snapshots_df[self.snapshots_df['variant'] == variant]
            
            # Calculate correlation matrix for the three metrics
            correlation_matrix = variant_data[[
                'warehouse_utilization',
                'planned_moves',
                'arrival_queue_size'
            ]].corr()
            
            print(f"\nVariant: {self._format_variant_name(variant)}")
            print("\nCorrelation Matrix:")
            print(correlation_matrix.round(3))
            print("\n" + "-" * 50)

    def plot_compact_correlations(self):
        """Create a single, compact correlation comparison across variants"""
        # Get variants in original order from configs
        variants = [config['variant'] for config in self.raw_data['configs']]
        
        # Create a DataFrame to store all correlations
        correlation_data = []
        metric_pairs = [
            ('warehouse_utilization', 'planned_moves', 'Utilization-Planned Moves'),
            ('warehouse_utilization', 'arrival_queue_size', 'Utilization-Arrival Queue'),
            ('planned_moves', 'arrival_queue_size', 'Planned Moves-Arrival Queue')
        ]
        
        for variant in variants:
            variant_data = self.snapshots_df[self.snapshots_df['variant'] == variant]
            corr_matrix = variant_data[[
                'warehouse_utilization',
                'planned_moves',
                'arrival_queue_size'
            ]].corr()
            
            for metric1, metric2, pair_name in metric_pairs:
                correlation_data.append({
                    'Variant': self._format_variant_name(variant),
                    'Metric Pair': pair_name,
                    'Correlation': corr_matrix.loc[metric1, metric2]
                })
        
        df_correlations = pd.DataFrame(correlation_data)
        
        # Create the visualization
        plt.figure(figsize=(10, 6))
        
        # Create heatmap
        pivot_table = df_correlations.pivot(
            index='Metric Pair', 
            columns='Variant', 
            values='Correlation'
        )
        
        sns.heatmap(
            pivot_table,
            annot=True,
            fmt='.3f',
            cmap='Reds',  # Using Reds colormap but keeping full scale
            center=0,
            vmin=-1,
            vmax=1,
            cbar_kws={'label': 'Correlation Coefficient'}
        )
        
        plt.title('Correlation Analysis Across Variants', pad=20)
        plt.xticks(rotation=30, ha='right')
        plt.tight_layout()
        
        return plt.gcf()

    def analyze_correlations_compact(self, save_path=None):
        """Generate and save compact correlation analysis"""
        fig = self.plot_compact_correlations()
        
        if save_path:
            fig.savefig(save_path / 'correlation_analysis_compact.png', 
                    dpi=300, bbox_inches='tight', pad_inches=0.3)
            plt.close(fig)
        else:
            plt.show()

# Add this to your existing analysis method
def analyze_correlations(self, save_path=None):
    """Calculate correlations and optionally save visualization"""
    # Print numerical results
    print("\nCorrelation Analysis:")
    print("-" * 50)
    
    variants = self.snapshots_df['variant'].unique()
    for variant in variants:
        variant_data = self.snapshots_df[self.snapshots_df['variant'] == variant]
        correlation_matrix = variant_data[[
            'warehouse_utilization',
            'planned_moves',
            'arrival_queue_size'
        ]].corr()
        
        print(f"\nVariant: {self._format_variant_name(variant)}")
        print("\nCorrelation Matrix:")
        print(correlation_matrix.round(3))
        print("\n" + "-" * 50)
    
    # Create and save visualization
    fig = self.plot_correlation_heatmap()
    if save_path:
        fig.savefig(save_path / 'correlation_analysis.png', 
                   dpi=300, bbox_inches='tight', pad_inches=0.3)
        plt.close(fig)
    else:
        plt.show()
    

def generate_visualizations(results_path: str, output_dir: str = 'visualization_output'):
    analyzer = BlockRelocationAnalyzer(results_path)
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    analyzer.plot_analysis(output_path)
    analyzer.generate_summary_stats(output_path)
    analyzer.analyze_correlations()
    analyzer.plot_compact_correlations()

    return analyzer

In [62]:
class RobustnessAnalyzer(BlockRelocationAnalyzer):
    def __init__(self, results_paths: List[str]):
        """Initialize with multiple result files from different seeds"""
        # Initialize with first file to get warehouse info and configs
        with open(results_paths[0], 'r') as f:
            first_data = json.load(f)
            self.warehouse_info = first_data['warehouse']
            self.raw_data = {
                'warehouse': self.warehouse_info,
                'configs': first_data['configs'],
                'runs': []
            }
        
        # Collect runs from all files
        for path in results_paths:
            with open(path, 'r') as f:
                data = json.load(f)
                self.raw_data['runs'].extend(data['runs'])
        
        self.warehouse_str = f"Warehouse {self.warehouse_info['dimensions']}"
        self.snapshots_df = self._process_snapshots()
        self.runs_df = self._process_runs()
        
        # Initialize plotting style from parent class
        plt.style.use('seaborn-v0_8-whitegrid')
        plt.rcParams.update({
            'font.family': 'serif',
            'font.size': 10,
            'figure.figsize': [20, 20],
            'axes.titlesize': 11,
            'axes.labelsize': 10,
            'legend.fontsize': 9,
            'legend.framealpha': 1.0,
            'legend.facecolor': 'white',
            'legend.edgecolor': 'gray'
        })

    def plot_robustness_analysis(self, save_path: Path = None):
        """Create improved robustness analysis visualizations"""
        fig = plt.figure(figsize=(30, 25))
        gs = plt.GridSpec(4, 2, figure=fig, hspace=0.5, wspace=0.4)
        
        # Get variants in original order from config
        variants = [config['variant'] for config in self.raw_data['configs']]
        
        # Set up consistent colors and markers for variants
        colors = plt.cm.Set2(np.linspace(0, 1, len(variants)))
        markers = ['o', 's', '^', 'D', 'v', 'p', '*']
        variant_styles = {
            variant: {
                'color': color,
                'marker': marker
            }
            for variant, color, marker in zip(variants, colors, markers)
        }

        # 1. Success Rate by Variant with improved colors and order
        ax1 = fig.add_subplot(gs[0, 0])
        success_rates = self.runs_df.groupby('variant')['status'].value_counts(normalize=True).unstack()
        success_rates = success_rates.reindex(variants)
        
        # Reorder columns to put Terminated in middle and Failed on top
        status_colors = {
            'Failed': '#e74c3c',  # Red
            'Terminated': '#f1c40f',  # Yellow
            'Completed': '#2ecc71'  # Green
        }
        status_order = ['Failed', 'Terminated', 'Completed']
        success_rates = success_rates[status_order]
        
        success_rates.plot(
            kind='bar',
            stacked=True,
            ax=ax1,
            color=[status_colors[x] for x in status_order]
        )
        
        # Add percentage labels on bars
        for c in ax1.containers:
            # Convert to percentages
            labels = [f'{(v*100):.0f}%' if v > 0 else '' for v in c.datavalues]
            ax1.bar_label(c, labels=labels, label_type='center')
            
        ax1.set_title('Success Rate by Variant Across All Seeds')
        ax1.set_xlabel('Variant')
        ax1.set_ylabel('Percentage')
        ax1.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax1.get_xticklabels()],
            rotation=30,
            ha='right'
        )
        ax1.legend(title='Status', bbox_to_anchor=(1.02, 1))

        # 2. Total Events Distribution
        ax2 = fig.add_subplot(gs[0, 1])
        total_events = self.runs_df.groupby(['variant', 'seed'])['events_reached'].mean().reset_index()
        # Force the order of variants
        total_events['variant'] = pd.Categorical(total_events['variant'], categories=variants, ordered=True)
        sns.boxplot(
            data=total_events,
            x='variant',
            y='events_reached',
            ax=ax2,
            order=variants,
            palette={v: style['color'] for v, style in variant_styles.items()}
        )
        ax2.set_title('Total Events Reached Distribution by Variant')
        ax2.set_xlabel('Variant')
        ax2.set_ylabel('Total Events Reached')
        ax2.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax2.get_xticklabels()],
            rotation=30,
            ha='right'
        )

        # 3. Normalized Performance Plot
        ax3 = fig.add_subplot(gs[1, 0])
        completed_runs = self.runs_df[self.runs_df['status'] == 'Completed']
        
        # Calculate normalized metrics per seed
        normalized_metrics = []
        for seed in completed_runs['seed'].unique():
            seed_data = completed_runs[completed_runs['seed'] == seed]
            if not seed_data.empty:
                # Normalize runtime and cost by the minimum value for that seed
                min_runtime = seed_data['runtime'].min()
                min_cost = seed_data['total_cost'].min()
                
                for _, row in seed_data.iterrows():
                    normalized_metrics.append({
                        'variant': row['variant'],
                        'seed': seed,
                        'normalized_runtime': row['runtime'] / min_runtime,
                        'normalized_cost': row['total_cost'] / min_cost
                    })
        
        norm_df = pd.DataFrame(normalized_metrics)
        
        # Plot normalized metrics
        for variant in variants:
            variant_data = norm_df[norm_df['variant'] == variant]
            if not variant_data.empty:
                style = variant_styles[variant]
                ax3.scatter(
                    variant_data['normalized_runtime'],
                    variant_data['normalized_cost'],
                    color=style['color'],
                    marker=style['marker'],
                    label=self._format_variant_name(variant),
                    alpha=0.7,
                    s=100
                )
        
        ax3.set_title('Normalized Performance (Relative to Best in Each Seed)')
        ax3.set_xlabel('Normalized Runtime (1.0 = Best)')
        ax3.set_ylabel('Normalized Cost (1.0 = Best)')
        ax3.plot([1, ax3.get_xlim()[1]], [1, 1], 'k--', alpha=0.3)  # Horizontal reference
        ax3.plot([1, 1], [1, ax3.get_ylim()[1]], 'k--', alpha=0.3)  # Vertical reference
        ax3.legend(bbox_to_anchor=(1.02, 1))

        # 4. Relative Performance Heatmap with improved explanation
        ax4 = fig.add_subplot(gs[1, 1])
        completed_variants = completed_runs['variant'].unique()
        seed_performance = completed_runs.pivot_table(
            values=['runtime', 'total_cost'],
            index='seed',
            columns='variant',
            aggfunc='mean'
        )
        
        # Normalize each metric within seeds
        relative_cost = seed_performance['total_cost'].div(seed_performance['total_cost'].mean(axis=1), axis=0)
        relative_runtime = seed_performance['runtime'].div(seed_performance['runtime'].mean(axis=1), axis=0)
        
        # Combine metrics (you could weight them differently if desired)
        relative_performance = (relative_cost + relative_runtime) / 2
        
        relative_performance = relative_performance[variants]  # Reorder columns
        
        sns.heatmap(
            relative_performance,
            ax=ax4,
            cmap='RdYlBu_r',
            center=1,
            annot=True,
            fmt='.2f',
            cbar_kws={'label': 'Relative Performance\n(Lower is Better)'}
        )
        ax4.set_title('Relative Performance by Seed\n(Combined Runtime and Cost, Normalized per Seed)')
        ax4.set_xlabel('Variant')
        ax4.set_ylabel('Seed')
        ax4.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax4.get_xticklabels()],
            rotation=30,
            ha='right'
        )

        # 5. Stability Score Plot
        ax5 = fig.add_subplot(gs[2, 0])
        
        # Calculate stability scores
        stability_scores = {}
        for variant in completed_variants:
            variant_data = completed_runs[completed_runs['variant'] == variant]
            
            # Calculate coefficient of variation for key metrics
            runtime_cv = (variant_data['runtime'].std() / variant_data['runtime'].mean()) * 100
            cost_cv = (variant_data['total_cost'].std() / variant_data['total_cost'].mean()) * 100
            events_cv = (variant_data['events_reached'].std() / variant_data['events_reached'].mean()) * 100
            
            # Combined stability score (lower is better)
            stability_scores[variant] = {
                'Runtime CV': runtime_cv,
                'Cost CV': cost_cv,
                'Events CV': events_cv
            }
        
        stability_df = pd.DataFrame(stability_scores).T
        stability_df.plot(
            kind='bar',
            ax=ax5,
            width=0.8
        )
        ax5.set_title('Stability Analysis (Lower CV = More Stable)')
        ax5.set_xlabel('Variant')
        ax5.set_ylabel('Coefficient of Variation (%)')
        ax5.set_xticklabels(
            [self._format_variant_name(x) for x in stability_df.index],
            rotation=30,
            ha='right'
        )
        ax5.legend(bbox_to_anchor=(1.02, 1))

        # 6. Success Rate vs Performance Plot
        ax6 = fig.add_subplot(gs[2, 1])
        
        # Calculate success rate and average normalized performance
        performance_metrics = []
        for variant in variants:
            variant_runs = self.runs_df[self.runs_df['variant'] == variant]
            success_rate = (variant_runs['status'] == 'Completed').mean() * 100
            
            variant_norm = norm_df[norm_df['variant'] == variant]
            if not variant_norm.empty:
                avg_norm_performance = (variant_norm['normalized_runtime'] + 
                                     variant_norm['normalized_cost']).mean() / 2
            else:
                avg_norm_performance = np.nan
                
            performance_metrics.append({
                'variant': variant,
                'success_rate': success_rate,
                'avg_normalized_performance': avg_norm_performance
            })
        
        perf_df = pd.DataFrame(performance_metrics)
        
        for variant in variants:
            variant_data = perf_df[perf_df['variant'] == variant]
            if not variant_data.empty and not np.isnan(variant_data['avg_normalized_performance'].iloc[0]):
                style = variant_styles[variant]
                ax6.scatter(
                    variant_data['success_rate'],
                    variant_data['avg_normalized_performance'],
                    color=style['color'],
                    marker=style['marker'],
                    s=200,
                    label=self._format_variant_name(variant)
                )
                
                # Add variant labels
                ax6.annotate(
                    self._format_variant_name(variant),
                    (variant_data['success_rate'].iloc[0],
                     variant_data['avg_normalized_performance'].iloc[0]),
                    xytext=(5, 5),
                    textcoords='offset points'
                )
        
        ax6.set_title('Success Rate vs Average Performance')
        ax6.set_xlabel('Success Rate (%)')
        ax6.set_ylabel('Avg Normalized Performance\n(Lower is Better)')
        ax6.legend(bbox_to_anchor=(1.02, 1))

        # Adjust layout
        plt.tight_layout(rect=[0, 0, 0.85, 1])
        
        if save_path:
            plt.savefig(save_path / 'robustness_analysis.png', dpi=300, bbox_inches='tight', pad_inches=0.3)
            plt.close()
        else:
            plt.show()

    def plot_enhanced_robustness_analysis(self, save_path: Path = None):
        """Create enhanced robustness analysis visualizations with separate metric heatmaps"""
        # Set up the figure
        fig = plt.figure(figsize=(25, 30))
        gs = plt.GridSpec(4, 2, figure=fig, hspace=0.4, wspace=0.3)
        
        # Get variants in original order from config
        variants = [config['variant'] for config in self.raw_data['configs']]
        
        # 1. Runtime Heatmap
        ax1 = fig.add_subplot(gs[0, 0])
        completed_runs = self.runs_df[self.runs_df['status'] == 'Completed']
        runtime_pivot = completed_runs.pivot_table(
            values='runtime',
            index='seed',
            columns='variant',
            aggfunc='mean'
        )
        # Normalize runtime within seeds
        runtime_relative = runtime_pivot.div(runtime_pivot.mean(axis=1), axis=0)
        runtime_relative = runtime_relative[variants]  # Reorder columns
        
        sns.heatmap(
            runtime_relative,
            ax=ax1,
            cmap='RdYlBu_r',
            center=1,
            annot=True,
            fmt='.2f',
            cbar_kws={'label': 'Relative Runtime\n(Lower is Better)'}
        )
        ax1.set_title('Relative Runtime by Seed')
        ax1.set_xlabel('Variant')
        ax1.set_ylabel('Seed')
        ax1.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax1.get_xticklabels()],
            rotation=30,
            ha='right'
        )

        # 2. Cost Heatmap
        ax2 = fig.add_subplot(gs[0, 1])
        cost_pivot = completed_runs.pivot_table(
            values='total_cost',
            index='seed',
            columns='variant',
            aggfunc='mean'
        )
        cost_relative = cost_pivot.div(cost_pivot.mean(axis=1), axis=0)
        cost_relative = cost_relative[variants]
        
        sns.heatmap(
            cost_relative,
            ax=ax2,
            cmap='RdYlBu_r',
            center=1,
            annot=True,
            fmt='.2f',
            cbar_kws={'label': 'Relative Cost\n(Lower is Better)'}
        )
        ax2.set_title('Relative Cost by Seed')
        ax2.set_xlabel('Variant')
        ax2.set_ylabel('Seed')
        ax2.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax2.get_xticklabels()],
            rotation=30,
            ha='right'
        )

        # 3. Average Warehouse Utilization Heatmap (Absolute Values, Completed Runs Only)
        ax3 = fig.add_subplot(gs[1, 0])
        
        # Get successful runs
        completed_runs = self.runs_df[self.runs_df['status'] == 'Completed']
        completed_pairs = set(zip(completed_runs['seed'], completed_runs['variant']))
        
        # Filter snapshots to only include data from completed runs
        completed_snapshots = self.snapshots_df[
            self.snapshots_df.apply(lambda row: (row['seed'], row['variant']) in completed_pairs, axis=1)
        ]
        
        # Calculate average warehouse utilization for completed runs
        util_data = completed_snapshots.groupby(['seed', 'variant'])['warehouse_utilization'].mean().reset_index()
        util_pivot = util_data.pivot(index='seed', columns='variant', values='warehouse_utilization')
        util_pivot = util_pivot[variants]  # Reorder columns
        
        # Use YlOrRd colormap for utilization (yellow=low, red=high)
        sns.heatmap(
            util_pivot * 100,  # Convert to percentage
            ax=ax3,
            cmap='YlOrRd',
            annot=True,
            fmt='.1f',
            cbar_kws={'label': 'Warehouse Utilization (%)'}
        )
        ax3.set_title('Average Warehouse Utilization by Seed (%)')
        ax3.set_xlabel('Variant')
        ax3.set_ylabel('Seed')
        ax3.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax3.get_xticklabels()],
            rotation=30,
            ha='right'
        )

        # 4. Events Reached Heatmap
        ax4 = fig.add_subplot(gs[1, 1])
        events_pivot = self.runs_df.pivot_table(
            values='events_reached',
            index='seed',
            columns='variant',
            aggfunc='mean'
        )
        events_relative = events_pivot.div(events_pivot.mean(axis=1), axis=0)
        events_relative = events_relative[variants]
        
        sns.heatmap(
            events_relative,
            ax=ax4,
            cmap='RdYlBu_r',
            center=1,
            annot=True,
            fmt='.2f',
            cbar_kws={'label': 'Relative Events Reached\n(Higher is Better)'}
        )
        ax4.set_title('Relative Events Reached by Seed')
        ax4.set_xlabel('Variant')
        ax4.set_ylabel('Seed')
        ax4.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax4.get_xticklabels()],
            rotation=30,
            ha='right'
        )

        # 5. Time to Failure/Termination Heatmap
        ax5 = fig.add_subplot(gs[2, 0])
        failed_terminated = self.runs_df[self.runs_df['status'].isin(['Failed', 'Terminated'])]
        if not failed_terminated.empty:
            failure_pivot = failed_terminated.pivot_table(
                values='events_reached',
                index='seed',
                columns='variant',
                aggfunc='mean'
            )
            failure_relative = failure_pivot.div(failure_pivot.mean(axis=1), axis=0)
            failure_relative = failure_relative[variants]
            
            sns.heatmap(
                failure_relative,
                ax=ax5,
                cmap='RdYlBu_r',
                center=1,
                annot=True,
                fmt='.2f',
                cbar_kws={'label': 'Relative Time to Failure\n(Higher is Better)'}
            )
            ax5.set_title('Relative Time to Failure/Termination by Seed')
            ax5.set_xlabel('Variant')
            ax5.set_ylabel('Seed')
            ax5.set_xticklabels(
                [self._format_variant_name(x.get_text()) for x in ax5.get_xticklabels()],
                rotation=30,
                ha='right'
            )

        # 6. Success Rate Plot (kept from original)
        ax6 = fig.add_subplot(gs[2, 1])
        success_rates = self.runs_df.groupby('variant')['status'].value_counts(normalize=True).unstack()
        success_rates = success_rates.reindex(variants)
        
        status_colors = {
            'Failed': '#e74c3c',     # Red
            'Terminated': '#f1c40f',  # Yellow
            'Completed': '#2ecc71'    # Green
        }
        status_order = ['Failed', 'Terminated', 'Completed']
        success_rates = success_rates[status_order]
        
        success_rates.plot(
            kind='bar',
            stacked=True,
            ax=ax6,
            color=[status_colors[x] for x in status_order]
        )
        
        # Add percentage labels on bars
        for c in ax6.containers:
            labels = [f'{(v*100):.0f}%' if v > 0 else '' for v in c.datavalues]
            ax6.bar_label(c, labels=labels, label_type='center')
            
        ax6.set_title('Success Rate by Variant')
        ax6.set_xlabel('Variant')
        ax6.set_ylabel('Percentage')
        ax6.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax6.get_xticklabels()],
            rotation=30,
            ha='right'
        )
        ax6.legend(title='Status', bbox_to_anchor=(1.02, 1))

        # Add summary statistics table
        ax7 = fig.add_subplot(gs[3, :])
        ax7.axis('off')
        
        # Calculate summary statistics
        summary_data = []
        for variant in variants:
            variant_data = self.runs_df[self.runs_df['variant'] == variant]
            completed_data = variant_data[variant_data['status'] == 'Completed']
            
            summary = {
                'Variant': self._format_variant_name(variant),
                'Success Rate': f"{(len(completed_data) / len(variant_data) * 100):.1f}%",
                'Avg Runtime': f"{completed_data['runtime'].mean():.1f}s" if not completed_data.empty else 'N/A',
                'Avg Cost': f"{completed_data['total_cost'].mean():.0f}" if not completed_data.empty else 'N/A',
                'Avg Events': f"{variant_data['events_reached'].mean():.0f}",
                'Avg Utilization': f"{self.snapshots_df[self.snapshots_df['variant'] == variant]['warehouse_utilization'].mean()*100:.1f}%"
            }
            summary_data.append(summary)
        
        summary_df = pd.DataFrame(summary_data)
        table = ax7.table(
            cellText=summary_df.values,
            colLabels=summary_df.columns,
            loc='center',
            cellLoc='center'
        )
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1.2, 1.5)
        
        # Adjust layout
        plt.tight_layout(rect=[0, 0, 0.95, 0.95])
        
        if save_path:
            plt.savefig(save_path / 'enhanced_robustness_analysis.png', 
                        dpi=300, bbox_inches='tight', pad_inches=0.3)
            plt.close()
        else:
            plt.show()

    def generate_robustness_summary(self, output_path: Path):
        """Generate improved robustness statistics"""
        # Status distribution
        status_dist = pd.crosstab(
            self.runs_df['variant'],
            self.runs_df['status'],
            normalize='index'
        ).round(3) * 100

        # Performance metrics for completed runs only
        completed_runs = self.runs_df[self.runs_df['status'] == 'Completed']
        
        # Calculate normalized metrics per seed
        normalized_metrics = []
        for seed in completed_runs['seed'].unique():
            seed_data = completed_runs[completed_runs['seed'] == seed]
            if not seed_data.empty:
                min_runtime = seed_data['runtime'].min()
                min_cost = seed_data['total_cost'].min()
                
                for _, row in seed_data.iterrows():
                    normalized_metrics.append({
                        'variant': row['variant'],
                        'seed': seed,
                        'normalized_runtime': row['runtime'] / min_runtime,
                        'normalized_cost': row['total_cost'] / min_cost
                    })
        
        norm_df = pd.DataFrame(normalized_metrics)
        
        # Calculate summary statistics
        performance_stats = pd.DataFrame()
        
        # Success rate
        success_rate = (self.runs_df['status'] == 'Completed').groupby(self.runs_df['variant']).mean() * 100
        performance_stats['Success Rate (%)'] = success_rate
        
        # Average normalized performance (when completed)
        avg_norm_performance = norm_df.groupby('variant').agg({
            'normalized_runtime': ['mean', 'std'],
            'normalized_cost': ['mean', 'std']
        })
        
        performance_stats['Avg Normalized Runtime'] = avg_norm_performance['normalized_runtime']['mean']
        performance_stats['Runtime Stability (CV%)'] = (
            avg_norm_performance['normalized_runtime']['std'] / 
            avg_norm_performance['normalized_runtime']['mean'] * 100
        )
        
        performance_stats['Avg Normalized Cost'] = avg_norm_performance['normalized_cost']['mean']
        performance_stats['Cost Stability (CV%)'] = (
            avg_norm_performance['normalized_cost']['std'] / 
            avg_norm_performance['normalized_cost']['mean'] * 100
        )
        
        # Average events reached (all runs)
        events_stats = self.runs_df.groupby('variant')['events_reached'].agg([
            'mean', 'std', 'min', 'max'
        ]).round(2)
        
        performance_stats['Avg Events Reached'] = events_stats['mean']
        
        # Format the index to be more readable
        performance_stats.index = [self._format_variant_name(v) for v in performance_stats.index]
        
        # Save to CSV
        performance_stats.to_csv(output_path / 'robustness_summary.csv')
        
        return performance_stats

In [ ]:
class WarehouseAnalyzer:
    def __init__(self, results_paths_by_size):
        """Initialize with dictionary mapping warehouse sizes to result file paths
        
        Args:
            results_paths_by_size (dict): Dictionary mapping warehouse dimensions (str) to result file path
        """
        self.analyzers_by_size = {}
        # Keep original order of dimensions
        self.dimensions = list(results_paths_by_size.keys())
        
        # Create BlockRelocationAnalyzer for each warehouse size (single file version)
        for dim, paths in results_paths_by_size.items():
            # Take first (and only) path from the list
            self.analyzers_by_size[dim] = BlockRelocationAnalyzer(paths[0])
            
        # Get variants in original order from configs
        self.variants = [config['variant'] for config in 
                        list(self.analyzers_by_size.values())[0].raw_data['configs']]
        
        # Set up consistent plotting style
        plt.style.use('seaborn-v0_8-whitegrid')
        # Import matplotlib ticker for log scale formatting
        from matplotlib import ticker
        plt.rcParams.update({
            'font.family': 'serif',
            'font.size': 10,
            'figure.figsize': [20, 10],
            'axes.titlesize': 12,
            'axes.labelsize': 10,
            'legend.fontsize': 9,
            'legend.framealpha': 1.0,
            'legend.facecolor': 'white',
            'legend.edgecolor': 'gray'
        })

    def _format_variant_name(self, name: str) -> str:
        """Format variant names for better readability"""
        words = []
        current = ""
        for char in name:
            if char.isupper() and current:
                words.append(current)
                current = char
            else:
                current += char
        words.append(current)
        return ' '.join(words)

    def plot_warehouse_analysis(self, save_path: Path = None):
        """Create visualization comparing variant performance across warehouse sizes"""
        fig = plt.figure(figsize=(29, 23))
        gs = plt.GridSpec(3, 2, figure=fig, hspace=0.4, wspace=0.3)

        # Set up consistent colors and markers for variants
        colors = plt.cm.Set2(np.linspace(0, 1, len(self.variants)))
        markers = ['o', 's', '^', 'D', 'v', 'p', '*']
        variant_styles = {
            variant: {
                'color': color,
                'marker': marker
            }
            for variant, color, marker in zip(self.variants, colors, markers)
        }

        # 1. Success Rate Heatmap
        ax1 = fig.add_subplot(gs[0, 0])
        success_data = []
        
        for dim in self.dimensions:  # Use stored order
            analyzer = self.analyzers_by_size[dim]
            for variant in self.variants:  # Use original variant order
                variant_runs = analyzer.runs_df[analyzer.runs_df['variant'] == variant]
                success_rate = (variant_runs['status'] == 'Completed').mean() * 100
                success_data.append({
                    'Warehouse': dim,
                    'Variant': variant,
                    'Success Rate': success_rate
                })
        
        success_df = pd.DataFrame(success_data)
        success_pivot = success_df.pivot(
            index='Warehouse',
            columns='Variant',
            values='Success Rate'
        )
        # Ensure correct column order
        success_pivot = success_pivot[self.variants]
        
        sns.heatmap(
            success_pivot,
            ax=ax1,
            cmap='RdYlGn',
            annot=True,
            fmt='.1f',
            vmin=0,
            vmax=100,
            cbar_kws={'label': 'Success Rate (%)'}
        )
        ax1.set_title('Success Rate by Warehouse Size')
        ax1.set_xlabel('Variant')
        ax1.set_ylabel('Warehouse Dimensions')
        ax1.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax1.get_xticklabels()],
            rotation=30,
            ha='right'
        )

        # 2. Runtime Analysis
        ax2 = fig.add_subplot(gs[0, 1])
        runtime_data = []
        
        for dim in self.dimensions:
            analyzer = self.analyzers_by_size[dim]
            completed_runs = analyzer.runs_df[analyzer.runs_df['status'] == 'Completed']
            
            for variant in self.variants:
                variant_runs = completed_runs[completed_runs['variant'] == variant]
                if not variant_runs.empty:
                    runtime_data.append({
                        'Warehouse': dim,
                        'Variant': variant,
                        'Runtime': variant_runs['runtime'].mean()
                    })
        
        runtime_df = pd.DataFrame(runtime_data)
        # Ensure warehouse dimension ordering
        runtime_df['Warehouse'] = pd.Categorical(runtime_df['Warehouse'], categories=self.dimensions, ordered=True)
        runtime_df = runtime_df.sort_values('Warehouse')
        
        for variant in self.variants:
            variant_data = runtime_df[runtime_df['Variant'] == variant]
            if not variant_data.empty:
                style = variant_styles[variant]
                # Get indices for the dimensions present in variant_data
                x_indices = [self.dimensions.index(dim) for dim in variant_data['Warehouse']]
                ax2.plot(
                    x_indices,
                    variant_data['Runtime'],
                    label=self._format_variant_name(variant),
                    color=style['color'],
                    marker=style['marker'],
                    markersize=10,
                    linewidth=2
                )
                
        ax2.set_yscale('log')  # Set log scale for runtime
        ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: f'{x:.1f}s'))  # Format y-axis labels
        ax2.grid(True, which="both", ls="-", alpha=0.2)  # Add grid for both major and minor ticks
        
        # Add minor grid lines
        ax2.grid(True, which="minor", ls=":", alpha=0.1)
        
        ax2.set_title('Average Runtime by Warehouse Size\n(Completed Runs Only)')
        ax2.set_xlabel('Warehouse Dimensions')
        ax2.set_xticks(range(len(self.dimensions)))
        ax2.set_xticklabels(self.dimensions)
        ax2.set_ylabel('Runtime (seconds)')
        ax2.legend(bbox_to_anchor=(1.02, 1))
        ax2.grid(True)

        # 3. Cost Analysis
        ax3 = fig.add_subplot(gs[1, 0])
        cost_data = []
        
        for dim in self.dimensions:
            analyzer = self.analyzers_by_size[dim]
            completed_runs = analyzer.runs_df[analyzer.runs_df['status'] == 'Completed']
            
            for variant in self.variants:
                variant_runs = completed_runs[completed_runs['variant'] == variant]
                if not variant_runs.empty:
                    cost_data.append({
                        'Warehouse': dim,
                        'Variant': variant,
                        'Cost': variant_runs['total_cost'].mean()
                    })
        
        cost_df = pd.DataFrame(cost_data)
        # Ensure warehouse dimension ordering
        cost_df['Warehouse'] = pd.Categorical(cost_df['Warehouse'], categories=self.dimensions, ordered=True)
        cost_df = cost_df.sort_values('Warehouse')
        
        for variant in self.variants:
            variant_data = cost_df[cost_df['Variant'] == variant]
            if not variant_data.empty:
                style = variant_styles[variant]
                # Get indices for the dimensions present in variant_data
                x_indices = [self.dimensions.index(dim) for dim in variant_data['Warehouse']]
                ax3.plot(
                    x_indices,
                    variant_data['Cost'],
                    label=self._format_variant_name(variant),
                    color=style['color'],
                    marker=style['marker'],
                    markersize=10,
                    linewidth=2
                )
        
        ax3.set_title('Average Cost by Warehouse Size\n(Completed Runs Only)')
        ax3.set_xlabel('Warehouse Dimensions')
        ax3.set_xticks(range(len(self.dimensions)))
        ax3.set_xticklabels(self.dimensions)
        ax3.set_ylabel('Total Cost')
        ax3.legend(bbox_to_anchor=(1.02, 1))
        ax3.grid(True)

        # 4. Warehouse Utilization
        ax4 = fig.add_subplot(gs[1, 1])
        utilization_data = []
        
        for dim in self.dimensions:
            analyzer = self.analyzers_by_size[dim]
            # Only consider snapshots from completed runs
            completed_runs = analyzer.runs_df[analyzer.runs_df['status'] == 'Completed']
            completed_pairs = set(zip(completed_runs['seed'], completed_runs['variant']))
            
            completed_snapshots = analyzer.snapshots_df[
                analyzer.snapshots_df.apply(lambda row: 
                    (row['seed'], row['variant']) in completed_pairs, axis=1)
            ]
            
            for variant in self.variants:
                variant_snapshots = completed_snapshots[
                    completed_snapshots['variant'] == variant]
                if not variant_snapshots.empty:
                    utilization_data.append({
                        'Warehouse': dim,
                        'Variant': variant,
                        'Utilization': variant_snapshots['warehouse_utilization'].mean() * 100
                    })
        
        util_df = pd.DataFrame(utilization_data)
        # Ensure warehouse dimension ordering
        util_df['Warehouse'] = pd.Categorical(util_df['Warehouse'], categories=self.dimensions, ordered=True)
        util_df = util_df.sort_values('Warehouse')
        
        for variant in self.variants:
            variant_data = util_df[util_df['Variant'] == variant]
            if not variant_data.empty:
                style = variant_styles[variant]
                # Get indices for the dimensions present in variant_data
                x_indices = [self.dimensions.index(dim) for dim in variant_data['Warehouse']]
                ax4.plot(
                    x_indices,
                    variant_data['Utilization'],
                    label=self._format_variant_name(variant),
                    color=style['color'],
                    marker=style['marker'],
                    markersize=10,
                    linewidth=2
                )
        
        ax4.set_title('Average Warehouse Utilization\n(Completed Runs Only)')
        ax4.set_xlabel('Warehouse Dimensions')
        ax4.set_xticks(range(len(self.dimensions)))
        ax4.set_xticklabels(self.dimensions)
        ax4.set_ylabel('Utilization (%)')
        ax4.legend(bbox_to_anchor=(1.02, 1))
        ax4.grid(True)

        # 5. Events Reached Analysis
        ax5 = fig.add_subplot(gs[2, 0])
        events_data = []
        
        for dim in self.dimensions:
            analyzer = self.analyzers_by_size[dim]
            for variant in self.variants:
                variant_runs = analyzer.runs_df[analyzer.runs_df['variant'] == variant]
                events_data.append({
                    'Warehouse': dim,
                    'Variant': variant,
                    'Events': variant_runs['events_reached_percentage'].mean()
                })
        
        events_df = pd.DataFrame(events_data)
        # Ensure warehouse dimension ordering
        events_df['Warehouse'] = pd.Categorical(events_df['Warehouse'], categories=self.dimensions, ordered=True)
        events_df = events_df.sort_values('Warehouse')
        
        for variant in self.variants:
            variant_data = events_df[events_df['Variant'] == variant]
            if not variant_data.empty:
                style = variant_styles[variant]
                # Get indices for the dimensions present in variant_data
                x_indices = [self.dimensions.index(dim) for dim in variant_data['Warehouse']]
                ax5.plot(
                    x_indices,
                    variant_data['Events'],
                    label=self._format_variant_name(variant),
                    color=style['color'],
                    marker=style['marker'],
                    markersize=10,
                    linewidth=2
                )
        
        ax5.set_title('Average Events Reached (%)\n(All Runs)')
        ax5.set_xlabel('Warehouse Dimensions')
        ax5.set_xticks(range(len(self.dimensions)))
        ax5.set_xticklabels(self.dimensions)
        ax5.set_ylabel('Events Reached (%)')
        ax5.legend(bbox_to_anchor=(1.02, 1))
        ax5.grid(True)

        # 6. Status Distribution
        ax6 = fig.add_subplot(gs[2, 1])
        status_data = []
        
        for dim in self.dimensions:
            analyzer = self.analyzers_by_size[dim]
            status_dist = pd.crosstab(
                analyzer.runs_df['variant'],
                analyzer.runs_df['status'],
                normalize='index'
            ) * 100
            
            for variant in self.variants:
                if variant in status_dist.index:
                    row = status_dist.loc[variant]
                    status_data.append({
                        'Warehouse': dim,
                        'Variant': self._format_variant_name(variant),
                        'Completed': row.get('Completed', 0),
                        'Failed': row.get('Failed', 0),
                        'Terminated': row.get('Terminated', 0)
                    })
        
        status_df = pd.DataFrame(status_data)
        
        # Create the grouped bar plot
        bar_width = 0.15
        r = np.arange(len(self.dimensions))
        
        for i, variant in enumerate(self.variants):
            variant_data = status_df[status_df['Variant'] == self._format_variant_name(variant)]
            style = variant_styles[variant]
            position = r + i * bar_width
            
            ax6.bar(position, 
                   variant_data['Completed'],
                   bar_width,
                   label=self._format_variant_name(variant),
                   color=style['color'],
                   alpha=0.7)
        
        ax6.set_title('Success Rate Distribution')
        ax6.set_xlabel('Warehouse Dimensions')
        ax6.set_ylabel('Success Rate (%)')
        ax6.set_xticks(r + bar_width * (len(self.variants) - 1) / 2)
        ax6.set_xticklabels(self.dimensions)
        ax6.legend(bbox_to_anchor=(1.02, 1))
        ax6.grid(True)

        # Adjust layout
        plt.tight_layout(rect=[0, 0, 0.9, 1])
        
        if save_path:
            plt.savefig(save_path / 'warehouse_analysis.png', 
                       dpi=300, bbox_inches='tight', pad_inches=0.3)
            plt.close()
        else:
            plt.show()

    def generate_warehouse_summary(self, output_path: Path):
        """Generate summary statistics for warehouse analysis"""
        summary_data = []
        
        for dim in self.dimensions:
            analyzer = self.analyzers_by_size[dim]
            
            for variant in self.variants:
                variant_runs = analyzer.runs_df[analyzer.runs_df['variant'] == variant]
                completed_runs = variant_runs[variant_runs['status'] == 'Completed']
                
                summary = {
                    'Warehouse': dim,
                    'Variant': self._format_variant_name(variant),
                    'Success Rate (%)': (len(completed_runs) / len(variant_runs) * 100).round(1),
                    'Avg Runtime (s)': completed_runs['runtime'].mean().round(2) if not completed_runs.empty else None,
                    'Avg Cost': completed_runs['total_cost'].mean().round(2) if not completed_runs.empty else None,
                    'Avg Events Reached (%)': (variant_runs['events_reached_percentage'].mean()).round(1),
                }
                
                # Add utilization if there are completed runs
                if not completed_runs.empty:
                    completed_pairs = set(zip(completed_runs['seed'], completed_runs['variant']))
                    completed_snapshots = analyzer.snapshots_df[
                        analyzer.snapshots_df.apply(lambda row: 
                            (row['seed'], row['variant']) in completed_pairs, axis=1)
                    ]
                    summary['Avg Utilization (%)'] = (
                        completed_snapshots['warehouse_utilization'].mean() * 100
                    ).round(1)
                else:
                    summary['Avg Utilization (%)'] = None
                
                summary_data.append(summary)
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv(output_path / 'warehouse_summary.csv', index=False)
        
        return summary_df

In [ ]:
class BeamWidthAnalyzer:
    def __init__(self, results_paths_by_beam: Dict[int, str]):
        """Initialize with dictionary mapping beam widths to result file paths
        
        Args:
            results_paths_by_beam (dict): Dictionary mapping beam widths (int) to result file path
        """
        self.analyzers_by_beam = {}
        # Keep original order of beam widths
        self.beam_widths = sorted(list(results_paths_by_beam.keys()))
        
        # Create BlockRelocationAnalyzer for each beam width
        for beam_width, path in results_paths_by_beam.items():
            self.analyzers_by_beam[beam_width] = BlockRelocationAnalyzer(path)
            
        # Get variants in original order from configs
        self.variants = [config['variant'] for config in 
                        list(self.analyzers_by_beam.values())[0].raw_data['configs']]
        
        # Set up consistent plotting style
        plt.style.use('seaborn-v0_8-whitegrid')
        plt.rcParams.update({
            'font.family': 'serif',
            'font.size': 10,
            'figure.figsize': [20, 20],
            'axes.titlesize': 12,
            'axes.labelsize': 10,
            'legend.fontsize': 9,
            'legend.framealpha': 1.0,
            'legend.facecolor': 'white',
            'legend.edgecolor': 'gray'
        })

    def _format_variant_name(self, name: str) -> str:
        """Format variant names for better readability"""
        words = []
        current = ""
        for char in name:
            if char.isupper() and current:
                words.append(current)
                current = char
            else:
                current += char
        words.append(current)
        return ' '.join(words)
    
    def plot_beam_width_analysis(self, save_path: Optional[Path] = None):
        """Create visualization comparing variant performance across beam widths"""
        fig = plt.figure(figsize=(30, 22))
        gs = plt.GridSpec(3, 2, figure=fig, hspace=0.4, wspace=0.3)

        # Set up consistent colors and markers for variants
        colors = plt.cm.Set2(np.linspace(0, 1, len(self.variants)))
        markers = ['o', 's', '^', 'D', 'v', 'p', '*']
        variant_styles = {
            variant: {
                'color': color,
                'marker': marker
            }
            for variant, color, marker in zip(self.variants, colors, markers)
        }

        # 1. Runtime Analysis by Beam Width
        ax1 = fig.add_subplot(gs[0, 0])
        runtime_data = []

        for beam_width in self.beam_widths:
            analyzer = self.analyzers_by_beam[beam_width]
            completed_runs = analyzer.runs_df[analyzer.runs_df['status'] == 'Completed']
            
            for variant in self.variants:
                variant_runs = completed_runs[completed_runs['variant'] == variant]
                if not variant_runs.empty:
                    runtime_data.append({
                        'Beam Width': beam_width,
                        'Variant': variant,
                        'Runtime': variant_runs['runtime'].mean()
                    })

        runtime_df = pd.DataFrame(runtime_data)

        for variant in self.variants:
            variant_data = runtime_df[runtime_df['Variant'] == variant]
            if not variant_data.empty:
                style = variant_styles[variant]
                ax1.plot(
                    variant_data['Beam Width'],
                    variant_data['Runtime'],
                    label=self._format_variant_name(variant),
                    color=style['color'],
                    marker=style['marker'],
                    markersize=10,
                    linewidth=2
                )

        ax1.set_title('Runtime vs Beam Width\n(Completed Runs Only)')
        ax1.set_xlabel('Beam Width')
        ax1.set_ylabel('Runtime (seconds)')
        # Set x-axis to log scale (beam width)
        ax1.set_xscale('log', base=2)
        # Set y-axis to log scale (runtime)
        ax1.set_yscale('log')
        # Format the y-axis tick labels for better readability
        ax1.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: f'{y:.1f}'))
        ax1.set_xticks(self.beam_widths)
        ax1.set_xticklabels(self.beam_widths)
        ax1.legend(bbox_to_anchor=(1.02, 1))
        ax1.grid(True, which='both', linestyle='--', linewidth=0.5)  # Grid lines for both major and minor ticks
        # 2. Cost Analysis by Beam Width
        ax2 = fig.add_subplot(gs[0, 1])
        cost_data = []
        
        for beam_width in self.beam_widths:
            analyzer = self.analyzers_by_beam[beam_width]
            completed_runs = analyzer.runs_df[analyzer.runs_df['status'] == 'Completed']
            
            for variant in self.variants:
                variant_runs = completed_runs[completed_runs['variant'] == variant]
                if not variant_runs.empty:
                    cost_data.append({
                        'Beam Width': beam_width,
                        'Variant': variant,
                        'Cost': variant_runs['total_cost'].mean()
                    })
        
        cost_df = pd.DataFrame(cost_data)
        
        for variant in self.variants:
            variant_data = cost_df[cost_df['Variant'] == variant]
            if not variant_data.empty:
                style = variant_styles[variant]
                ax2.plot(
                    variant_data['Beam Width'],
                    variant_data['Cost'],
                    label=self._format_variant_name(variant),
                    color=style['color'],
                    marker=style['marker'],
                    markersize=10,
                    linewidth=2
                )
        
        ax2.set_title('Cost vs Beam Width\n(Completed Runs Only)')
        ax2.set_xlabel('Beam Width')
        ax2.set_ylabel('Total Cost')
        ax2.set_xscale('log', base=2)  # Log scale for beam width
        ax2.set_xticks(self.beam_widths)
        ax2.set_xticklabels(self.beam_widths)
        ax2.legend(bbox_to_anchor=(1.02, 1))
        ax2.grid(True)

        # 3. Success Rate by Beam Width
        ax3 = fig.add_subplot(gs[1, 0])
        success_data = []
        
        for beam_width in self.beam_widths:
            analyzer = self.analyzers_by_beam[beam_width]
            for variant in self.variants:
                variant_runs = analyzer.runs_df[analyzer.runs_df['variant'] == variant]
                success_rate = (variant_runs['status'] == 'Completed').mean() * 100
                success_data.append({
                    'Beam Width': beam_width,
                    'Variant': variant,
                    'Success Rate': success_rate
                })
        
        success_df = pd.DataFrame(success_data)
        
        for variant in self.variants:
            variant_data = success_df[success_df['Variant'] == variant]
            if not variant_data.empty:
                style = variant_styles[variant]
                ax3.plot(
                    variant_data['Beam Width'],
                    variant_data['Success Rate'],
                    label=self._format_variant_name(variant),
                    color=style['color'],
                    marker=style['marker'],
                    markersize=10,
                    linewidth=2
                )
        
        ax3.set_title('Success Rate vs Beam Width')
        ax3.set_xlabel('Beam Width')
        ax3.set_ylabel('Success Rate (%)')
        ax3.set_xscale('log', base=2)  # Log scale for beam width
        ax3.set_xticks(self.beam_widths)
        ax3.set_xticklabels(self.beam_widths)
        ax3.set_ylim(0, 105)  # Set y-axis limit to ensure 100% is visible
        ax3.legend(bbox_to_anchor=(1.02, 1))
        ax3.grid(True)

        # 4. Success Rate Heatmap
        ax4 = fig.add_subplot(gs[1, 1])
        success_pivot = success_df.pivot(
            index='Beam Width',
            columns='Variant',
            values='Success Rate'
        )
        # Ensure correct column order
        success_pivot = success_pivot[self.variants]
        
        sns.heatmap(
            success_pivot,
            ax=ax4,
            cmap='RdYlGn',
            annot=True,
            fmt='.1f',
            vmin=0,
            vmax=100,
            cbar_kws={'label': 'Success Rate (%)'}
        )
        ax4.set_title('Success Rate Heatmap')
        ax4.set_xlabel('Variant')
        ax4.set_ylabel('Beam Width')
        ax4.set_xticklabels(
            [self._format_variant_name(x.get_text()) for x in ax4.get_xticklabels()],
            rotation=30,
            ha='right'
        )

        # 5. Runtime vs Cost Scatter Plot with Beam Width as color
        ax5 = fig.add_subplot(gs[2, 0])
        
        runtime_cost_data = []
        for beam_width in self.beam_widths:
            analyzer = self.analyzers_by_beam[beam_width]
            completed_runs = analyzer.runs_df[analyzer.runs_df['status'] == 'Completed']
            
            for variant in self.variants:
                variant_runs = completed_runs[completed_runs['variant'] == variant]
                if not variant_runs.empty:
                    runtime_cost_data.append({
                        'Beam Width': beam_width,
                        'Variant': variant,
                        'Runtime': variant_runs['runtime'].mean(),
                        'Cost': variant_runs['total_cost'].mean()
                    })
        
        runtime_cost_df = pd.DataFrame(runtime_cost_data)
        
        # Create a colormap for beam widths
        beam_cmap = plt.cm.viridis
        beam_norm = plt.Normalize(min(self.beam_widths), max(self.beam_widths))
        
        for variant in self.variants:
            variant_data = runtime_cost_df[runtime_cost_df['Variant'] == variant]
            if not variant_data.empty:
                style = variant_styles[variant]
                
                # Draw lines connecting points of the same variant

                
                # Plot points with beam width as color intensity
                for _, row in variant_data.iterrows():
                    ax5.scatter(
                        row['Runtime'],
                        row['Cost'],
                        color=style['color'],
                        marker=style['marker'],
                        s=130,
                        label=f"{self._format_variant_name(variant)} (Beam: {row['Beam Width']})" if _ == variant_data.index[0] else "",
                        alpha=0.8,
                        edgecolor='black',
                        linewidth=1
                    )
                    
                    # Add beam width text annotation
                    ax5.annotate(
                        str(int(row['Beam Width'])),
                        (row['Runtime'], row['Cost']),
                        textcoords="offset points", 
                        xytext=(0,7), 
                        ha='center',
                        fontsize=8,
                        bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
                    )
        
        ax5.set_title('Runtime vs Cost by Variant and Beam Width')
        ax5.set_xlabel('Runtime (seconds)')
        ax5.set_ylabel('Total Cost')
        
        # Add a legend for variants only (beam width is annotated on points)
        handles, labels = ax5.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax5.legend(by_label.values(), by_label.keys(), 
                   title="Variants (with Beam Width labels on points)",
                   bbox_to_anchor=(1.02, 1))
        ax5.grid(True)

        # 6. Performance Improvement Plot (runtime and cost decrease percentage)
        ax6 = fig.add_subplot(gs[2, 1])
        
        # Calculate performance improvement (compared to smallest beam width)
        improvement_data = []
        min_beam = min(self.beam_widths)
        
        # First, get baseline values for each variant with minimum beam width
        baselines = {}
        min_beam_analyzer = self.analyzers_by_beam[min_beam]
        min_beam_completed = min_beam_analyzer.runs_df[min_beam_analyzer.runs_df['status'] == 'Completed']
        
        for variant in self.variants:
            variant_data = min_beam_completed[min_beam_completed['variant'] == variant]
            if not variant_data.empty:
                baselines[variant] = {
                    'runtime': variant_data['runtime'].mean(),
                    'cost': variant_data['total_cost'].mean()
                }
        
        # Then calculate improvements for each beam width
        for beam_width in self.beam_widths:
            if beam_width == min_beam:
                continue  # Skip baseline
                
            analyzer = self.analyzers_by_beam[beam_width]
            completed_runs = analyzer.runs_df[analyzer.runs_df['status'] == 'Completed']
            
            for variant in self.variants:
                if variant not in baselines:
                    continue  # Skip if no baseline
                    
                variant_runs = completed_runs[completed_runs['variant'] == variant]
                if not variant_runs.empty:
                    runtime = variant_runs['runtime'].mean()
                    cost = variant_runs['total_cost'].mean()
                    
                    runtime_improvement = (baselines[variant]['runtime'] - runtime) / baselines[variant]['runtime'] * 100
                    cost_improvement = (baselines[variant]['cost'] - cost) / baselines[variant]['cost'] * 100
                    
                    improvement_data.append({
                        'Beam Width': beam_width,
                        'Variant': variant,
                        'Runtime Improvement (%)': runtime_improvement,
                        'Cost Improvement (%)': cost_improvement
                    })
        
        improvement_df = pd.DataFrame(improvement_data)
        
        # Plot runtime improvement
        for variant in self.variants:
            variant_data = improvement_df[improvement_df['Variant'] == variant]
            if not variant_data.empty:
                style = variant_styles[variant]
                ax6.plot(
                    variant_data['Beam Width'],
                    variant_data['Runtime Improvement (%)'],
                    label=f"{self._format_variant_name(variant)} - Runtime",
                    color=style['color'],
                    marker=style['marker'],
                    markersize=8,
                    linewidth=2,
                    linestyle='-'
                )
                
                # Plot cost improvement with dashed line
                ax6.plot(
                    variant_data['Beam Width'],
                    variant_data['Cost Improvement (%)'],
                    label=f"{self._format_variant_name(variant)} - Cost",
                    color=style['color'],
                    marker='x',
                    markersize=8,
                    linewidth=2,
                    linestyle='--'
                )
        
        ax6.set_title(f'Performance Improvement vs Beam Width\n(Compared to Beam={min_beam})')
        ax6.set_xlabel('Beam Width')
        ax6.set_ylabel('Improvement (%)')
        ax6.set_xscale('log', base=2)  # Log scale for beam width
        ax6.set_xticks(self.beam_widths[1:])  # Skip first beam width as it's the baseline
        ax6.set_xticklabels(self.beam_widths[1:])
        ax6.axhline(y=0, color='k', linestyle='-', alpha=0.3)  # Add horizontal line at y=0
        ax6.legend(bbox_to_anchor=(1.02, 1), fontsize=8)
        ax6.grid(True)

        # Adjust layout
        plt.tight_layout(rect=[0, 0, 0.85, 1])
        
        if save_path:
            plt.savefig(save_path / 'beam_width_analysis.png', 
                       dpi=300, bbox_inches='tight', pad_inches=0.3)
            plt.close()
        else:
            plt.show()
        
        return fig
    
    def generate_beam_width_summary(self, output_path: Path):
        """Generate summary statistics for beam width analysis"""
        summary_data = []
        
        for beam_width in self.beam_widths:
            analyzer = self.analyzers_by_beam[beam_width]
            
            for variant in self.variants:
                variant_runs = analyzer.runs_df[analyzer.runs_df['variant'] == variant]
                completed_runs = variant_runs[variant_runs['status'] == 'Completed']
                
                summary = {
                    'Beam Width': beam_width,
                    'Variant': self._format_variant_name(variant),
                    'Success Rate (%)': (len(completed_runs) / len(variant_runs) * 100).round(1),
                    'Avg Runtime (s)': completed_runs['runtime'].mean().round(2) if not completed_runs.empty else None,
                    'Avg Cost': completed_runs['total_cost'].mean().round(2) if not completed_runs.empty else None,
                    'Avg Events Reached (%)': (variant_runs['events_reached_percentage'].mean()).round(1),
                }
                
                # Add warehouse utilization
                if not completed_runs.empty:
                    completed_pairs = set(zip(completed_runs['seed'], completed_runs['variant']))
                    completed_snapshots = analyzer.snapshots_df[
                        analyzer.snapshots_df.apply(lambda row: 
                            (row['seed'], row['variant']) in completed_pairs, axis=1)
                    ]
                    summary['Avg Utilization (%)'] = (
                        completed_snapshots['warehouse_utilization'].mean() * 100
                    ).round(1)
                else:
                    summary['Avg Utilization (%)'] = None
                
                summary_data.append(summary)
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv(output_path / 'beam_width_summary.csv', index=False)
        
        return summary_df

    def compute_optimal_beam_width(self, runtime_weight=0.5, cost_weight=0.5):
        """Compute the optimal beam width for each variant based on weighted runtime and cost
        
        Args:
            runtime_weight: Weight for runtime in the optimization (default: 0.5)
            cost_weight: Weight for cost in the optimization (default: 0.5)
            
        Returns:
            DataFrame with optimal beam width for each variant
        """
        # Normalize runtime and cost across all beam widths for each variant
        normalized_metrics = []
        
        for variant in self.variants:
            # Extract runtime and cost for all beam widths
            variant_metrics = []
            for beam_width in self.beam_widths:
                analyzer = self.analyzers_by_beam[beam_width]
                completed_runs = analyzer.runs_df[
                    (analyzer.runs_df['variant'] == variant) &
                    (analyzer.runs_df['status'] == 'Completed')
                ]
                
                if not completed_runs.empty:
                    variant_metrics.append({
                        'Beam Width': beam_width,
                        'Runtime': completed_runs['runtime'].mean(),
                        'Cost': completed_runs['total_cost'].mean(),
                        'Success Rate': (len(completed_runs) / len(analyzer.runs_df[analyzer.runs_df['variant'] == variant])) * 100
                    })
            
            if not variant_metrics:
                continue
                
            # Convert to DataFrame for easier processing
            metrics_df = pd.DataFrame(variant_metrics)
            
            # Skip if we don't have at least two beam widths
            if len(metrics_df) <= 1:
                continue
                
            # Min-max normalization of runtime and cost
            min_runtime = metrics_df['Runtime'].min()
            max_runtime = metrics_df['Runtime'].max()
            min_cost = metrics_df['Cost'].min()
            max_cost = metrics_df['Cost'].max()
            
            # Avoid division by zero
            runtime_range = max_runtime - min_runtime
            cost_range = max_cost - min_cost
            
            # Calculate normalized values (0 is best, 1 is worst)
            for _, row in metrics_df.iterrows():
                normalized_runtime = 0 if runtime_range == 0 else (row['Runtime'] - min_runtime) / runtime_range
                normalized_cost = 0 if cost_range == 0 else (row['Cost'] - min_cost) / cost_range
                
                # Calculate weighted score (lower is better)
                weighted_score = (runtime_weight * normalized_runtime) + (cost_weight * normalized_cost)
                
                normalized_metrics.append({
                    'Variant': variant,
                    'Beam Width': row['Beam Width'],
                    'Normalized Runtime': normalized_runtime,
                    'Normalized Cost': normalized_cost,
                    'Weighted Score': weighted_score,
                    'Success Rate': row['Success Rate'],
                    'Runtime': row['Runtime'],
                    'Cost': row['Cost']
                })
        
        norm_df = pd.DataFrame(normalized_metrics)
        
        # Find optimal beam width for each variant (minimum weighted score)
        optimal_beam = []
        for variant in self.variants:
            variant_data = norm_df[norm_df['Variant'] == variant]
            if not variant_data.empty:
                # Find the optimal beam width with at least 95% success rate
                reliable_beams = variant_data[variant_data['Success Rate'] >= 95]
                
                if not reliable_beams.empty:
                    optimal = reliable_beams.loc[reliable_beams['Weighted Score'].idxmin()]
                else:
                    # If no beam has 95% success, just find the minimum score
                    optimal = variant_data.loc[variant_data['Weighted Score'].idxmin()]
                
                optimal_beam.append({
                    'Variant': self._format_variant_name(variant),
                    'Optimal Beam Width': optimal['Beam Width'],
                    'Success Rate (%)': optimal['Success Rate'],
                    'Runtime (s)': optimal['Runtime'],
                    'Cost': optimal['Cost'],
                    'Score': optimal['Weighted Score']
                })
        
        optimal_df = pd.DataFrame(optimal_beam)
        return optimal_df


results_by_beam = {
    2: "results/beam_width_2_results.json",
    5: "results/beam_width_5_results.json", 
    10: "results/beam_width_10_results.json",
    50: "results/beam_width_50_results.json"
}

analyzer = BeamWidthAnalyzer(results_by_beam)

# Generate visualizations
analyzer.plot_beam_width_analysis(save_path=Path("output"))

# Generate summary statistics
analyzer.generate_beam_width_summary(output_path=Path("output"))

# Find optimal beam width
optimal_beam = analyzer.compute_optimal_beam_width(runtime_weight=0.3, cost_weight=0.7)
print(optimal_beam)

In [64]:
#generate_visualizations("results/standard_results.json")

# Beam width analysis
'''
analyze_beam_width(
    "results/beam_width_{}_results.json",
    beam_widths=[2, 4, 8, 16]
)

# Warehouse scaling analysis
analyze_warehouse_scaling({
    "5x6x3": "results/warehouse_5x6x3.json",
    "6x8x3": "results/warehouse_6x8x3.json",
    "8x10x3": "results/warehouse_8x10x3.json"
})

# Robustness analysis with multiple seeds
analyze_robustness(
    "results/seed_{}_results.json",
    seeds=[1337, 1338, 1339, 1340, 1341]
)
'''

'''
#robustness
seeds = [1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346]  # Add your seeds here
base_path = './results/seed_{}_results.json'  # Update this path pattern

# Create list of file paths
result_files = [base_path.format(seed) for seed in seeds]

# Create output directory
output_dir = Path('robustness_analysis_output')
output_dir.mkdir(exist_ok=True)

# Initialize analyzer and generate visualizations
analyzer = RobustnessAnalyzer(result_files)
analyzer.plot_enhanced_robustness_analysis(save_path=output_dir)
analyzer.generate_robustness_summary(output_dir)
'''
'''
# Create dictionary mapping warehouse dimensions to result files
results_by_size = {
    "6x3x3": ["results/warehouse_6x3x3.json"],
    "7x3x3": ["results/warehouse_7x3x3.json"],
    "7x4x3": ["results/warehouse_7x4x3.json"],
    "7x4x4": ["results/warehouse_7x5x4.json"],
}

analyzer = WarehouseAnalyzer(results_by_size)

# Generate visualizations
analyzer.plot_warehouse_analysis(save_path=Path("output"))

# Generate summary statistics
summary = analyzer.generate_warehouse_summary(output_path=Path("output"))

'''


'\n# Create dictionary mapping warehouse dimensions to result files\nresults_by_size = {\n    "6x3x3": ["results/warehouse_6x3x3.json"],\n    "7x3x3": ["results/warehouse_7x3x3.json"],\n    "7x4x3": ["results/warehouse_7x4x3.json"],\n    "7x4x4": ["results/warehouse_7x5x4.json"],\n}\n\nanalyzer = WarehouseAnalyzer(results_by_size)\n\n# Generate visualizations\nanalyzer.plot_warehouse_analysis(save_path=Path("output"))\n\n# Generate summary statistics\nsummary = analyzer.generate_warehouse_summary(output_path=Path("output"))\n\n'